In [5]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
#from tabula import read_pdf

## Read HIC Raw Data (2007-2021) and Standardize Column Names

In [6]:
os.chdir('../SourceData/RawHIC')

In [7]:
hic_2012 = pd.read_csv('2012HIC_RawData.csv',dtype={'Geo Code': 'Int64'})
hic_2012.rename(columns = {'HUD CoC Number':'CoC', 'Program Name':'Project Name', 'Program Type':'Project Type'}, inplace = True)

hic_2013 = pd.read_csv('2013-Housing-Inventory-Count-Raw-File.csv',dtype={'Geo Code': int})
hic_2013.rename(columns = {'CoC':'CoC Name','HudNum':'CoC', 'year': 'Year','Program Name':'Project Name', 'Program Type':'Project Type','Year-Round Beds':'Total Year Round Beds', 'CH Beds':'Chronically Homeless Beds'}, inplace = True)

hic_2014 = pd.read_csv('2014-Housing-Inventory-Count-Raw-File.csv',dtype={'Geo Code': int})
hic_2014.rename(columns = {'CoC':'CoC Name', 'HudNum':'CoC','year':'Year','Year-Round Beds':'Total Year Round Beds','CH Beds':'Chronically Homeless Beds'}, inplace = True)

hic_2015 = pd.read_csv('2015-Housing-Inventory-Count-Raw-File.csv',dtype={'Geo Code': int})
hic_2015.rename(columns = {'CoC':'CoC Name','HudNum':'CoC','year':'Year','Year-Round Beds':'Total Year Round Beds','Program Name':'Project Name', 'Program Type':'Project Type','CH Beds':'Chronically Homeless Beds'}, inplace = True)

hic_2016 = pd.read_csv('2016-Housing-Inventory-Count-Raw-File.csv',dtype={'Geo Code': int})
hic_2016.rename(columns = {'CoC':'CoC Name', 'HudNum':'CoC','notes':'Notes','year':'Year','Year-Round Beds':'Total Year Round Beds','Program Name':'Project Name', 'Program Type':'Project Type'}, inplace = True)

hic_2017 = pd.read_csv('2017-Housing-Inventory-Count-Raw-File.csv',dtype={'sandyRelatedNote': str,'Geo Code':int})
hic_2017.rename(columns = {'CoC':'CoC Name', 'HudNum':'CoC','year':'Year','address1':'Address1','address2':'Address2','city':'City','zip':'Zip','Year-Round Beds':'Total Year Round Beds','Program Name':'Project Name', 'Program Type':'Project Type'}, inplace = True)
hic_2017['Notes'] = hic_2017.notes+". "+hic_2017.projectNotes

hic_2018 = pd.read_csv('2018-Housing-Inventory-County-RawFile.csv',dtype={'Target Population B': str,'address2':str,'sandyRelatedNote':str,'Geo Code':int})
hic_2018.rename(columns = {'CoC':'CoC Name', 'HudNum':'CoC','year':'Year','address1':'Address1','address2':'Address2','city':'City','zip':'Zip','Year-Round Beds':'Total Year Round Beds','Program Name':'Project Name', 'Program Type':'Project Type'}, inplace = True)
hic_2018['Notes'] = hic_2018.notes+". "+hic_2018.projectNotes

hic_2019 = pd.read_csv('2019-Housing-Inventory-County-RawFile.csv',dtype={'sandyRelatedNote': str,'Geo Code':int})
hic_2019.rename(columns = {'CoC':'CoC Name', 'HudNum':'CoC','year':'Year','address1':'Address1','address2':'Address2','city':'City','zip':'Zip','Year-Round Beds':'Total Year Round Beds','Program Name':'Project Name', 'Program Type':'Project Type'}, inplace = True)
hic_2019['Notes'] = hic_2019.notes+". "+hic_2019.projectNotes

hic_2020 = pd.read_csv('2020-HIC-Raw-File.csv',dtype={'sandyRelatedNote': str,'Geo Code': int})
hic_2020.rename(columns = {'CoC':'CoC Name', 'HudNum':'CoC','year':'Year','Year-Round Beds':'Total Year Round Beds','address1':'Address1','address2':'Address2','city':'City','zip':'Zip','Program Name':'Project Name', 'Program Type':'Project Type'}, inplace = True)
hic_2020['Notes'] = hic_2020.notes+". "+hic_2020.projectNotes

hic_2021 = pd.read_csv('2021-HIC-Counts-by-State.csv',dtype={'sandyRelatedNote': str,'Geo Code':'Int64'})
hic_2021.rename(columns = {'CoC':'CoC Name', 'HudNum':'CoC','year':'Year','Year-Round Beds':'Total Year Round Beds','address1':'Address1','address2':'Address2','city':'City','zip':'Zip','Program Name':'Project Name', 'Program Type':'Project Type'}, inplace = True)
hic_2021['Notes'] = hic_2021.notes+". "+hic_2021.projectNotes


Combine all raw files into one dataframe

In [ ]:
# all_hic_dfs = [hic_2012,
#               hic_2013,
#               hic_2014,
#               hic_2015,
#               hic_2016,
#               hic_2017,
#               hic_2018,
#               hic_2019,
#               hic_2020,
#               hic_2021]

In [8]:
hic_raw = pd.concat([hic_2012,
                     hic_2013,
                     hic_2014,
                     hic_2015,
                     hic_2016,
                     hic_2017,
                     hic_2018,
                     hic_2019,
                     hic_2020,
                     hic_2021], axis=0, ignore_index=True)

Only keep columns that have been standardized across all years

In [9]:
hic_raw.drop(hic_raw.columns.difference(['CoC',
                                    'Year',
                                    'Organization Name', 
                                    'Project Name',
                                    'Project Type',
                                    'Bed Type',
                                    'Inventory Type',
                                    'Geo Code',
                                    'Total Year Round Beds',
                                    'Total Seasonal Beds',
                                    'Total Beds',
                                    'PIT Count',
                                    'CoC Name',
                                    'Chronically Homeless Beds',
                                    'Availability Start Date',
                                    'Availability End Date',
                                    'Address1',
                                    'Address2',
                                    'City',
                                    'Zip',
                                    'Notes',
                                    'housingType',
                                    'HMIS Org ID',
                                    'HMIS Project ID',
                                    'HIC Date',
                                    'HMIS Participating']), 1, inplace=True)

/var/folders/wd/xqhn8ntj0s58hdf7xthrvnq40000gn/T/ipykernel_13905/61070260.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  hic_raw.drop(hic_raw.columns.difference(['CoC',


Add a state variable

In [10]:
hic_raw['State']=hic_raw.CoC.str.slice(0, 2)

In [ ]:
for i in hic_raw.columns:
    print(i)

In [ ]:
##cleaned COC 
cleanedCoC = []
for i in hic_raw.CoC:
    if(i == "CA-600"):
        cleanedCoC.append('CA-600')
    else:
        cleanedCoC.append('all')

In [ ]:
#hic_raw['coc'] = cleanedCoC

In [11]:
hic_raw.head()

,CoC Name,CoC,Year,Organization Name,Project Name,Project Type,Bed Type,Geo Code,Inventory Type,Total Year Round Beds,...,Address2,City,Zip,Notes,housingType,HMIS Org ID,HMIS Project ID,HIC Date,HMIS Participating,State
0,Alaska Balance of State CoC,AK-501,2012,Adocates f/Victims of Violnc,DV Shelter,ES,F,29261,C,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AK
1,Alaska Balance of State CoC,AK-501,2012,AHFC - Public Housing Div,VASH-Fairbanks,PSH,NaN,29090,N,52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AK
2,Alaska Balance of State CoC,AK-501,2012,AK Family Services,DV Shelter,ES,F,29170,C,32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AK
3,Alaska Balance of State CoC,AK-501,2012,AK Family Services,Saxton Shelter,ES,F,29170,C,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AK
4,Alaska Balance of State CoC,AK-501,2012,Arctic Women in Crisis,DV Shelter,ES,F,29185,C,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AK


## Adding County Names Column (from Geo Codes)

In [ ]:
os.chdir('../HIC_geocodes')

geocodes.pdf found at https://www.hud.gov/sites/dfiles/CPD/documents/FY-2021-GeoCodes-with-PPRN_Final.pdf

In [ ]:
#uncomment this line if geocode dataframe is not already saved
#tabula.io.convert_into(input_path = 'geocodes.pdf', output_path= "geocodes.csv",pages = 'all')

In [ ]:
geocodes = pd.read_csv('geocodes.csv')
geocodes.head()

In [ ]:
geocodes = geocodes[['Geographic','NAME']].drop([0])
geocodes

In [ ]:
geocodes = geocodes[geocodes.Geographic != 'Geographic']
geocodes = geocodes[geocodes.Geographic != 'Name']
geocodes = geocodes[geocodes.Geographic != 'Code']

In [ ]:
##create dictionary
geocodes.Geographic =geocodes.Geographic.astype('int64')
geocodes

In [ ]:
geocodes_dict = dict(zip(geocodes.Geographic,geocodes.NAME))

In [ ]:
hic_raw['County'] = hic_raw['Geo Code'].map(geocodes_dict)

Uncomment to Export Cleaned Dataset:

In [ ]:
#hic_raw.to_csv('hic_cleaned.csv')

In [ ]:
hic_raw.columns

### Exploring Correlation between PIT Counts and Total Beds

In [ ]:
LA_agg = pd.DataFrame(LA_hic.groupby('Year')[['Total Beds','PIT Count']].sum())
LA_agg

In [ ]:
sns.lineplot(data = LA_agg, x = LA_agg.index, y = 'Total Beds', label = 'Total Beds') #orange
sns.lineplot(data = LA_agg, x = LA_agg.index, y = 'PIT Count',label = 'PIT Count') #blue
plt.show()